# HW5 — Problem 2: Thermit (adiabatic final state and welding use)

Stagewise energy balance from 298 K:
1) Heat to 1809 K and melt Fe
2) Heat to 2325 K and melt Al₂O₃
3) Heat to 3343 K
4) Vaporize Fe if budget remains

Data (J/mol/K unless noted):
- ΔH₍₂₉₈₎ = −852300 J per mol Fe₂O₃ reacted
- cp functions for Fe phases and Al₂O₃ (s/l) as given
- Fe transitions: α→γ (670), γ→δ (840), δ→l (13770) J/mol
- Tm,Fe=1809 K, Tm,Al₂O₃=2325 K, Tb,Fe=3343 K, ΔH_b,Fe=340159 J/mol


In [1]:

import numpy as np

# Reaction budget:
dH298 = -852300.0  # J per mol Fe2O3

# Heat capacity functions (J/mol/K)
def cp_Al2O3_s(T): return 117.49 + 10.38e-3*T - 37.11e5/(T**2)
def cp_Al2O3_l(T): return 184.1
def cp_Fe_alpha(T): return 37.12 + 6.17e-3*T + 56.92/(T**2)
def cp_Fe_gamma(T): return 24.48 + 8.45e-3*T
def cp_Fe_delta(T): return 37.12 + 6.17e-3*T + 56.92/(T**2)
def cp_Fe_l(T):     return 41.84

# Transition enthalpies (J/mol)
H_a2g = 670.0
H_g2d = 840.0
H_mFe = 13770.0
Hb_Fe = 340159.0

# Transition temperatures (K)
T_a2g = 1187.0
T_g2d = 1664.0
Tm_Fe = 1809.0
Tm_Al = 2325.0
Tb_Fe = 3343.0

# Analytic integrals for cp = a + bT + c/T^2
def int_cp_abc(a,b,c,T1,T2):
    return a*(T2-T1) + 0.5*b*(T2**2 - T1**2) - c*(1.0/T2 - 1.0/T1)

def int_cp_Al2O3_s(T1,T2): return int_cp_abc(117.49, 10.38e-3, 37.11e5, T1,T2)
def int_cp_Al2O3_l(T1,T2): return 184.1*(T2-T1)

# Note: cp_Fe_alpha/delta include +56.92/T^2 in cp -> integral uses c = -56.92
def int_cp_Fe_alpha(T1,T2): return int_cp_abc(37.12, 6.17e-3, -56.92, T1,T2)
def int_cp_Fe_gamma(T1,T2): return int_cp_abc(24.48, 8.45e-3, 0.0, T1,T2)
def int_cp_Fe_delta(T1,T2): return int_cp_abc(37.12, 6.17e-3, -56.92, T1,T2)
def int_cp_Fe_l(T1,T2):     return 41.84*(T2-T1)

def enthalpy_heat_to_melt_Fe_one_mol():
    H = 0.0
    H += int_cp_Fe_alpha(298.0, T_a2g)
    H += H_a2g
    H += int_cp_Fe_gamma(T_a2g, T_g2d)
    H += H_g2d
    H += int_cp_Fe_delta(T_g2d, Tm_Fe)
    H += H_mFe
    return H

H_Fe_298_to_melt = enthalpy_heat_to_melt_Fe_one_mol()

# Stage 1
H_Al_298_to_1809 = int_cp_Al2O3_s(298.0, Tm_Fe)
H_2Fe_298_to_melt = 2.0*H_Fe_298_to_melt
H_used_1 = H_Al_298_to_1809 + H_2Fe_298_to_melt
H_remain = -dH298 - H_used_1
print(f"Stage 1 used: {H_used_1:,.0f} J; remaining: {H_remain:,.0f} J")

# Stage 2
H_m_Al = 107000.0
H_Al_1809_to_2325_s = int_cp_Al2O3_s(Tm_Fe, Tm_Al)
H_2Fe_liq_1809_to_2325 = 2.0*int_cp_Fe_l(Tm_Fe, Tm_Al)
H_used_2 = H_Al_1809_to_2325_s + H_m_Al + H_2Fe_liq_1809_to_2325
H_remain -= H_used_2
print(f"Stage 2 used: {H_used_2:,.0f} J; remaining: {H_remain:,.0f} J")

# Stage 3
H_Al_liq_2325_to_3343 = int_cp_Al2O3_l(Tm_Al, Tb_Fe)
H_2Fe_liq_2325_to_3343 = 2.0*int_cp_Fe_l(Tm_Al, Tb_Fe)
H_used_3 = H_Al_liq_2325_to_3343 + H_2Fe_liq_2325_to_3343

if H_used_3 <= H_remain:
    H_remain -= H_used_3
    print(f"Stage 3 used: {H_used_3:,.0f} J; remaining before vaporization: {H_remain:,.0f} J")
    n_vap = H_remain / Hb_Fe
    print(f"Final T = 3343 K; vaporized Fe = {n_vap:.3f} mol; Fe(l) = {2.0 - n_vap:.3f} mol; Al2O3(l) = 1.000 mol")
else:
    total_cp = 184.1 + 2.0*41.84
    dT = H_remain / total_cp
    Tfinal = Tm_Al + dT
    print(f"Final T between 2325 and 3343 K: {Tfinal:.1f} K; no vaporization")


Stage 1 used: 357,874 J; remaining: 494,426 J
Stage 2 used: 222,330 J; remaining: 272,096 J
Final T between 2325 and 3343 K: 3341.1 K; no vaporization
